# Graph network code for Jodie

In [172]:
# import pandas as pd
# import numpy as np
# import json

# df = pd.read_csv('./data/aruba/aruba.csv')

# with open('./data/aruba/aruba.json') as f:

#     jsonfile = json.load(f)

# # jsonfile


# print((df.values.shape))



# config = {"config": [

#     {"name": "Meal_Preparation", "id": 0},
#     {"name": "Relax", "id": 1},
#     {"name": "Eating", "id": 2},
#     {"name": "Work", "id": 3},
#     {"name": "Sleeping", "id": 4},
#     {"name": "Wash_Dishes", "id": 5},
#     {"name": "Bed_to_Toilet", "id": 6},
#     {"name": "Enter_Home", "id": 7},
#     {"name": "Leave_Home", "id": 8},
#     {"name": "Housekeeping", "id": 9},
#     {"name": "Respirate", "id": 10},
#     {"name": "Idle", "id": 11}
# ]
# }

# # for all the values that are 1 in the sensor
# # user_id = room_id (json)
# # item_id = sensor_id(json)
# # comma_separated_list_of_features = [SensorType(json), Spos(json), value(csv), time_of_the__day(csv), rtype, room_location ]
# # state_label = activity (from csv for that row)

# columns = df.columns[4:]
# rows = []
# for i, row in enumerate(df.values):

#     if i == 100:
#         break
#         # get names of column where value is 1
#     sensor_True_Col = np.nonzero(row[4:] == 1)[0]
#     if sensor_True_Col.size != 0:
#         # print(i)
#         for val in sensor_True_Col:
#             # print(columns[val])
#             sensorName = columns[val]
#             # print(row)
#             row_rBedPressureRightoom = [row_room for row_room in jsonfile['rooms_location'] if sensorName in row_room['sensorsInRoom']][
#                 0]
#             user_id = row_room['id']

#             row_sensor = [[row_sensor['id'], row_sensor['location'], row_sensor['name']] for row_sensor in
#                           jsonfile['sensor_location'] if row_sensor['name'] == sensorName]
#             item_id = row_sensor[0][0]
#             row_sensor_name = row_sensor[0][2]
#             sensor_type = [155 if row_sensor_name.startswith('D') else 13][0]
#             sensorPosition = row_sensor[0][1]
#             value = 1
#             timestamp = row[3]
#             state_label = [className['id'] for className in config['config'] if className['name'] == row[2]][0]
#             comma_separated_list_of_features = np.array(
#                 [sensor_type, sensorPosition[0], sensorPosition[1], value, timestamp, row_room['id'],
#                  ])

#             B = np.copy(np.array([user_id, item_id, timestamp, state_label, comma_separated_list_of_features]))
#             rows.append(B)

#     else:
#         state_label = [className['id'] for className in config['config'] if className['name'] == row[2]][0]
#         rows.append(np.array(
#             [user_id, item_id, row[3], state_label, np.array([sensor_type, sensorPosition[0], sensorPosition[1], 0, row_room['id']])]))

# wikiDataFrame = pd.DataFrame(rows, columns=['user_id', 'item_id', 'timestamp', 'state_label',
#                                             'comma_separated_list_of_features'])

# wikiDataFrame.to_csv('wikipedia.csv')





# Make Master CSV

In [9]:
import pandas as pd
import numpy as np
import json


houseC = pd.read_csv('./data/master/houseC.csv')
ordonezB = pd.read_csv('./data/master/ordonezB.csv')

with open('./data/master/master.json') as f:

    masterJson = json.load(f)

In [3]:
df = pd.DataFrame( )
for key in masterJson.keys():
        
    currentCSV = key + '.csv'
    currentCSV = pd.read_csv('./data/master/' + currentCSV)
    temp = currentCSV[['start', 'end', 'activity', 'time_of_the_day']].copy()
    
    for dictionary in masterJson[key]:
        genericColumn = list(dictionary.keys())[0]            
        specificColumn = dictionary[genericColumn][0]['name']            
        temp[genericColumn] = currentCSV[specificColumn]
    df = pd.concat([df, temp])    
    df = df.fillna(-1)
    df.iloc[:,4:] = df[df.columns[4:]].apply(pd.to_numeric, downcast='integer')    

    
# df.to_csv('../Research/data/master/master.csv', index=False)

BedPressureRight
BasinBathroomPIR
ToiletFlushSensor
ShowerBathroomPIR
FridgeMagnetic
GroceriesCupboard
Microwave
ToiletSeatPressureSensor
FrontDoorMagnetic
LivingRoomDoorPIR
KitchenRoomDoorPIR
BedRoomDoorPIR


# Getting Unique activities in CSV's


In [196]:
!pwd

/home/harsh/Downloads/Thesis/Graph/dgl/related_code/3-basics


In [199]:


# df = pd.read_csv('./data/houseC/houseC.csv')



# Making Nodes and Edges for graph through house json file

In [6]:
!pwd
import os
import json
import pandas as pd
import numpy as np
import json

file_names = ['houseA','houseB','houseC','ordonezA','ordonezB']

for file_name in file_names:
    
    json_file_name = file_name + '.json'
    csv_file_name = file_name + '.csv'
    
    json_file = os.path.join('../../data', file_name, json_file_name )
    csv_file = os.path.join('../../data', file_name, csv_file_name)

        
    with open(json_file) as f:
        json_file = json.load(f)


    csv_file = pd.read_csv(csv_file)

    locations = json_file['locations']

    sensors = json_file['sensors']

    def sensor_place_in_house_ID(col_name, location_id = 1):
        # Find sensor dict and take the location key, whose name matches with col names        
        location_id = [x['location'] for x in sensors if x['name'] == col_name][0]            
        if len(location_id) == 1:
            location_type = [x['type'] for x in locations if x['id'] == location_id[0]][0]
            return [rooms.index(location_type)]
        else:
            items =[]
            for item in location_id:
                location_type = [x['type'] for x in locations if x['id'] == item][0]        
                items.append(rooms.index(location_type))
            return items


    # get rooms as nodes first, by giving them id and make a dictionary of their features
    nodes_dictionary_list = []
    d = {}

    rooms = [name['type'] for name in locations]
    count  = 0
    # convert it into a dictionary list
    for i, node in enumerate(rooms):
        d = {}
        d['Id'] = count
        d['Object'] = node
        d['Value'] = -1
        d['place_in_house'] = count
        d['Type'] = 1
        count+= 1
        nodes_dictionary_list.append(d)


    # csv column names
    col = list(csv_file.columns[4:].values)
    
    for i, col_name in enumerate(col):
        Id = i
        Object = col_name.split('_')[0]
        Value = -1            
        place_in_house = sensor_place_in_house_ID(Object)
        Type = 0

        # Append to nodes_dictionary_list 

        for location_id in place_in_house:
            d = {}        
            d['Id'] = len(nodes_dictionary_list)
            d['Object'] = Object
            d['Value'] = -1    
            d['Type'] = 0    
            d['place_in_house'] = location_id        
            nodes_dictionary_list.append(d)

    # Adding time of the day as the last node

    d ={}
    d['Id'] = len(nodes_dictionary_list)
    d['Object'] = 'time_of_the_day'
    d['Value'] = -1    
    d['Type'] = -1    
    d['place_in_house'] = -1        

    nodes_dictionary_list.append(d)

    # print(list(nodes_dictionary_list))

    import csv

    toCSV = nodes_dictionary_list



    # json_file = os.path.join('../../data', file_name)


    keys = toCSV[0].keys()
    with open(os.path.join('../../data', file_name, 'nodes.csv'), 'w', newline='')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)



    # Write Edge.csv

    # Make a list of src and destination and write it via pandas
    Src = []
    Dst = []
    for node_dict in nodes_dictionary_list:
        if node_dict['Type'] == 0:
            # Birectional Edges
            Src.append(node_dict['Id'])
            Dst.append(node_dict['place_in_house'])

            Src.append(node_dict['place_in_house'])
            Dst.append(node_dict['Id'])


    # Connect rooms 

    for location_dict in locations:    
        src_location_id = rooms.index(location_dict['type'])

        reaches_list = location_dict['reaches']
        for reach_id in reaches_list:        
            Type = [x['type'] for x in locations if x['id'] ==  reach_id][0]
            dst_location_id = rooms.index(Type)        
            Src.append(src_location_id)
            Dst.append(dst_location_id)

    Src.append(nodes_dictionary_list[-1]['Id'])
    Dst.append(nodes_dictionary_list[-1]['Id'])

    df = pd.DataFrame({'Src' : Src,'Dst' : Dst})    
    df.to_csv(os.path.join('../../data', file_name, 'bidrectional_edges.csv'), index=False)


/home/harsh/Downloads/Thesis/Graph/dgl/related_code/3-basics


In [12]:
import pandas as pd
import numpy as np
import pandas as pd
import json

In [14]:
houseB = pd.read_csv('../../data/houseB/houseB.csv')

In [17]:
print(len(houseB.iloc[0, :]))

26


In [177]:
# (a) x <= 5
# (d) 60 < x <= 120 
# (f) 150 < x <= 660
# (b) 5 < x <= 30 
# (e) 120 < x <= 150 
# (g) x > 660
# (c) 30 < x <= 60

In [178]:
seq_length = 1
prev_index = 0
seq_list = []
for index, row in df[:-1].iterrows():
    
    prev_value = df.iloc[prev_index, 4:]
    prev_value = ''.join(map(str, prev_value.values))
    
    if prev_value == ''.join(map(str, df.iloc[index + 1, 4:].values)):
        seq_length += 1
    else:        
        seq_list.append(seq_length)
        seq_length = 1
        prev_index = index + 1                
    
#     if index == 1000:
#         break

# sum_list =  row to be choosen in new dataframe.
sum_list = []
for i in range(len(seq_list)):
    sum_list.append(sum(seq_list[:i + 1]) -1)
# print('final rows list  ',sum_list)

# Seq_list = unary representation for each unique
# print('seq list ',  seq_list)

truncatedDataframe = pd.DataFrame([])

for index, x in enumerate(sum_list):         
    truncatedDataframe = truncatedDataframe.append(df.iloc[sum_list[index] ,:])


# convert each value according to bin, 
bins_seq_list = np.copy(seq_list)
for i, x in enumerate(bins_seq_list):
    if x <= 5:
        bins_seq_list[i] = 1
    elif 5 < x <= 30:
        bins_seq_list[i] = 2
    elif 30 < x <= 60:
        bins_seq_list[i] = 3
    elif 60 < x <= 120:
        bins_seq_list[i] = 4
    elif 120 < x <= 150:
        bins_seq_list[i] = 5
    elif 150 < x <= 660:
        bins_seq_list[i] = 6
    elif x > 660:
        bins_seq_list[i] = 7
                


In [179]:
truncatedDataframe['one_hot_time_of_the_day'] = round(truncatedDataframe['time_of_the_day'] * 24)

truncatedDataframe['one_hot_time_of_the_day'] = truncatedDataframe['one_hot_time_of_the_day'].apply(lambda x: np.array(x))




In [180]:
max(bins_seq_list)

6

In [181]:

# convert to unary representation
final_unary_list = []

for i in range(len(bins_seq_list)):
    gn_list = 7 * [0]
    ones_list = np.ones(bins_seq_list[i], dtype = int)
    zero_list = np.zeros(7 - bins_seq_list[i], dtype = int)
    unary_list = np.append(ones_list , zero_list)
    final_unary_list.append(unary_list)
    
truncatedDataframe['repeated_activitiesUnary'] = final_unary_list
truncatedDataframe['repeated_activitiesUnary'] = truncatedDataframe['repeated_activitiesUnary'].apply(lambda x: np.array(x))

In [182]:
# truncatedDataframe.head()

In [183]:
# list(final_unary_list)

In [184]:
[{"name": activity, "id":idx} for idx, activity in enumerate(pd.unique( df['activity']))]

[{'name': 'leaveHouse', 'id': 0},
 {'name': 'idle', 'id': 1},
 {'name': 'prepareDinner', 'id': 2},
 {'name': 'eatDinner', 'id': 3},
 {'name': 'getDrink', 'id': 4},
 {'name': 'washDishes', 'id': 5},
 {'name': 'brushTeeth', 'id': 6},
 {'name': 'goToBed', 'id': 7},
 {'name': 'useToilet', 'id': 8},
 {'name': 'prepareBreakfast', 'id': 9},
 {'name': 'eatBreakfast', 'id': 10},
 {'name': 'takeShower', 'id': 11},
 {'name': 'getDressed', 'id': 12}]

In [185]:
final_one_hot_time_of_day = []
for val in truncatedDataframe['one_hot_time_of_the_day']:        
    a = np.zeros(25, dtype = int)
    a[int(val)] =  1
    final_one_hot_time_of_day.append(a)
    

In [186]:
type(final_one_hot_time_of_day[0][0])

numpy.int64

In [187]:
truncatedDataframe['one_hot_time_of_the_day'] = final_one_hot_time_of_day

In [188]:
# Shuffling the column to restore the original order
col = truncatedDataframe.columns.values.tolist()

cols = ['start', 'end', 'activity', 'time_of_the_day']

col = [x for x in col if x not in cols]
col.insert(0, 'start')
col.insert(1, 'end')
col.insert(2, 'activity')
col.insert(3, 'time_of_the_day')

truncatedDataframe = truncatedDataframe[col]

In [194]:
truncatedDataframe.iloc[:, 4:26] = truncatedDataframe.iloc[:, 4:26].apply(pd.to_numeric, downcast='integer')

In [195]:
truncatedDataframe.to_pickle('../../Thesis/dummyTestforPycharm/data/houseC/one_hot')

truncatedDataframe.to_csv('../../Thesis/dummyTestforPycharm/data/houseC/one_hot.csv')

In [196]:
print(truncatedDataframe['one_hot_time_of_the_day'])

0        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
21       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
22       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
25       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                               ...                        
20546    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20558    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20559    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20572    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20573    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: one_hot_time_of_the_day, Length: 2148, dtype: object


In [197]:
truncatedDataframe.dtypes

start                         object
end                           object
activity                      object
time_of_the_day              float64
BalconyDoor_24                  int8
BathroomPIR_19                  int8
BedPressureLeft_13              int8
BedPressureRight_12             int8
BedroomDoor_10                  int8
BedroomDresser_18               int8
BedroomMovement_16              int8
CutleryDrawer_14                int8
Fridge_3                        int8
Frontdoor_7                     int8
GroceriesCupboard_5             int8
KitchenPIR_28                   int8
KitchenWindow_25                int8
Microwave_27                    int8
PianoSeatPressure_20            int8
PlatesCupboard_9                int8
ServerNookSeatPressure_22       int8
SinkFloat_21                    int8
StoveLid_15                     int8
Toaster_26                      int8
ToiletDoor_1                    int8
ToiletFlush_6                   int8
one_hot_time_of_the_day       object
r